# Background Service Worker

In [1]:
!pip install pymongo
!pip install dnspython

Import python packages.

In [1]:
from urllib import request
from pymongo import MongoClient
import json
from datetime import datetime
import time
import os
import sys

Get mongoDB database URL.

In [2]:
authFile = open(os.path.join('.', 'authfile.json'), 'r')
authFileContents = ''

for line in authFile:
    authFileContents = authFileContents + line

authParams = json.loads(authFileContents)

mongoHost = str(authParams['mongoHost'])
mongoUsername = str(authParams['mongoUsername'])
mongoPassword = str(authParams['mongoPassword'])
mongoDbName = str(authParams['mongoDbName'])

mongoCollectionName = 'sensorData'

Get API call URL details.

In [3]:
apiHost = 'swghosh.pythonanywhere.com'
apiPort = 80
apiURI = '/gdgu-mcmp/api/sensorData'

mongoURL = 'mongodb+srv://' + mongoUsername + ':' + mongoPassword + '@' + mongoHost + '/' + mongoDbName

# http urls that contain port 80 explicitly in it fails when urllib.request.urlopen(URL), appropriate fix
if not apiPort == 80:
    apiURL = 'http://' + apiHost + ':' + str(apiPort) + apiURI
else:
    apiURL = 'http://' + apiHost + apiURI

Set time frequency in seconds.

In [4]:
timeFrequency = 5

Continously run the background service.

In [ ]:
while True:
    try:
        client = MongoClient(mongoURL)

        db = client[mongoDbName]

        jsonRecvd = request.urlopen(apiURL).read().decode('utf-8')

        sensorData = json.loads(jsonRecvd)

        db[mongoCollectionName].insert_many(sensorData)

        client.close()

        print(str(datetime.now()) + ' - - [inserted to db collection] ' + str(sensorData))

    except KeyboardInterrupt:
        pass
    except Exception as exception:
        print(str(datetime.now()) + ' - - [error occured] ' + '\nRetrying in ' + str(timeFrequency) + ' seconds.', exception, file = sys.stderr)
    time.sleep(timeFrequency)

2018-09-02 21:04:16.776207 - - [inserted to db collection] [{'timestamp': '2018-09-02T15:34:05.127835', 'latitude': 28.2699415, 'longitude': 77.0636767, 'sensorChannel': {'name': 'GROUND-QUAL-SENSORS', 'id': 'g-channel'}, 'sensorData': {'soilMoist': {'value': 39, 'unit': 'mg/dL'}}, '_id': ObjectId('5b8c02ed5ccb0b3a63fef1e1')}, {'timestamp': '2018-09-02T15:34:05.127971', 'latitude': 28.2699415, 'longitude': 77.0636767, 'sensorChannel': {'name': 'AIR-QUAL-SENSORS', 'id': 'h-channel'}, 'sensorData': {'coTwoConc': {'value': 691, 'unit': 'mg/dL'}}, '_id': ObjectId('5b8c02ed5ccb0b3a63fef1e2')}]
2018-09-02 21:04:35.207042 - - [inserted to db collection] [{'timestamp': '2018-09-02T15:34:22.467790', 'latitude': 28.2699415, 'longitude': 77.0636767, 'sensorChannel': {'name': 'GROUND-QUAL-SENSORS', 'id': 'g-channel'}, 'sensorData': {'soilMoist': {'value': 70, 'unit': 'mg/dL'}}, '_id': ObjectId('5b8c02ff5ccb0b3a63fef1e4')}, {'timestamp': '2018-09-02T15:34:22.467816', 'latitude': 28.2699415, 'longit

Service should keep logging sensorData.